In [15]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None  # default='warn'
from datetime import datetime
now = datetime.now()
print(now)

2023-02-21 08:35:55.813888


#### Nguyên phụ liệu

In [16]:
dfProductUnit = pd.read_csv('../dataDA/ProductUnit.csv')
dfProductUnit

,product_id,product_unit_id,unit_id,name_unit,is_default
0,21,21,1,Mét,1
1,22,22,1,Mét,1
2,26,26,1,Mét,1
3,28,28,1,Mét,1
4,31,31,1,Mét,1
...,...,...,...,...,...
16842,7909,7919,20,Pia,1
16843,7914,7924,21,Cây,1
16844,7916,7926,21,Cây,1
16845,7917,7927,21,Cây,1


In [3]:
dfProductUnitDefault = dfProductUnit[dfProductUnit['is_default'] == 1]
dfProductUnitDefault

,product_id,product_unit_id,unit_id,name_unit,is_default
0,21,21,1,Mét,1
1,22,22,1,Mét,1
2,26,26,1,Mét,1
3,28,28,1,Mét,1
4,31,31,1,Mét,1
...,...,...,...,...,...
16842,7909,7919,20,Pia,1
16843,7914,7924,21,Cây,1
16844,7916,7926,21,Cây,1
16845,7917,7927,21,Cây,1


In [4]:
dfOrder = pd.read_csv('../dataDA/Product.csv')
dfOrder

,name,product_id,is_outsourcing,product_code,quantity_product
0,Lưới màn 80G Black,1,0,1-LUOIMA-BLK-80-CM,6301.91
1,Lưới màn 80G White,2,0,1-LUOIMA-WHI-80-CM,13742.73
2,Lưới màn 80G Light White,3,0,1-LUOIMA-LWH-80-CM,4437.70
3,Lưới màn mỏng 28G Black,4,0,1-LUOIMA-BLK-28-CM,3270.90
4,Lưới màn mỏng 28G White,5,0,1-LUOIMA-WHI-28-CM,2324.08
...,...,...,...,...,...
11155,Vải nylon Black ép mút 5mm,11468,1,1-NY0000-BLK-M5-PE,0.00
11156,Vải nylon Black ép mút 2mm,11469,1,1-NY0000-BLK-M2-PE,0.00
11157,Dây khóa da Black W160mm x H18mm HM-221126-190...,11470,0,6-KDHM06-BLK-DA-TQ,126.00
11158,Khuôn Dây khóa da Black W160mm x H18mm HM-2211...,11471,0,9-KDHM06-BLK-DA-TQ,1.00


In [5]:
dfProduct = pd.read_csv('../dataDA/Product.csv')
dfProductUnit = dfProduct.merge(dfProductUnitDefault , on = 'product_id' , how = 'left').drop(columns= {'product_unit_id' , 'unit_id' , 'is_default'})
dfProductUnit

,name,product_id,is_outsourcing,product_code,quantity_product,name_unit
0,Lưới màn 80G Black,1,0,1-LUOIMA-BLK-80-CM,6301.91,Mét
1,Lưới màn 80G White,2,0,1-LUOIMA-WHI-80-CM,13742.73,Mét
2,Lưới màn 80G Light White,3,0,1-LUOIMA-LWH-80-CM,4437.70,Mét
3,Lưới màn mỏng 28G Black,4,0,1-LUOIMA-BLK-28-CM,3270.90,Mét
4,Lưới màn mỏng 28G White,5,0,1-LUOIMA-WHI-28-CM,2324.08,Mét
...,...,...,...,...,...,...
11155,Vải nylon Black ép mút 5mm,11468,1,1-NY0000-BLK-M5-PE,0.00,Mét
11156,Vải nylon Black ép mút 2mm,11469,1,1-NY0000-BLK-M2-PE,0.00,Mét
11157,Dây khóa da Black W160mm x H18mm HM-221126-190...,11470,0,6-KDHM06-BLK-DA-TQ,126.00,Cái
11158,Khuôn Dây khóa da Black W160mm x H18mm HM-2211...,11471,0,9-KDHM06-BLK-DA-TQ,1.00,Cái


In [6]:
dfRelationship = pd.read_csv('../dataDA/Relationship.csv')
dfRelationship

,product_id,outsourcing_product_id,product_code,quota_outsourcing
0,73,75,1-CVAC04-SCA-00-IW,1.0
1,73,78,1-CVAC04-SCA-00-IW,1.0
2,73,79,1-CVAC04-SCA-00-IW,1.0
3,80,81,1-CVAC05-DBN-00-IW,1.0
4,96,101,1-CVAC07-GRY-00-HV,1.0
...,...,...,...,...
4826,11355,11412,1-P2585Q-CTG-00-EV,1.0
4827,8311,11412,1-GOLDAI-WHI-00-EP,1.0
4828,5,11412,1-LUOIMA-WHI-28-CM,1.0
4829,777,11468,1-NY0000-BLK-00-PE,1.0


In [7]:
dfProductUnit.to_excel('../dataDA/ProductUnit{}.xlsx'.format(now), index= False)

#### Production order + Stock Out

In [8]:
dfProductionOrder = pd.read_csv('../dataDA/ProductionOrderDetailBao.csv')
dfProductionOrder

,product_code,order_id,production_order_id,production_code,quantity_production_order,quantity_exported_production_order,time_create,completed_at,time_stock_out
0,1-CVAC04-SCA-00-HV,1,15,PYC-07-CAT,19.548,18.548,2022-05-13,2022-05-14,2022-05-16
1,1-CVAC12-ROL-00-IW,1,15,PYC-07-CAT,37.246,36.246,2022-05-13,2022-05-14,2022-05-16
2,2-M83200-BLK-00-DI,1,15,PYC-07-CAT,16.500,16.500,2022-05-13,2022-05-14,2022-05-16
3,7-CM3822-RED-50-PP,1,17,PYC-07-THU-DUC,1.500,1.500,2022-05-13,2022-05-16,2022-05-25
4,7-CM7382-BLU-50-PP,1,17,PYC-07-THU-DUC,4.000,4.000,2022-05-13,2022-05-16,2022-05-25
...,...,...,...,...,...,...,...,...,...
39970,3-NH3930-BLK-23-CY,2515,2413,PYC-863-PHOI,101.000,NaN,2022-12-05,2022-12-06,NaN
39971,9-GLNDYC-000-IN-TL,2515,2413,PYC-863-PHOI,100.000,NaN,2022-12-05,2022-12-06,NaN
39972,4-KATE00-BLK-N0-28,2515,2413,PYC-863-PHOI,148.500,NaN,2022-12-05,2022-12-06,NaN
39973,1-SUEDE0-BLK-L2-TQ,2515,2416,PYC-901-CAT,9.840,9.840,2022-12-05,2022-12-06,2022-12-05


In [9]:
dfOrder = pd.read_csv('../dataDA/OrderBao.csv')

In [10]:
dfProductionOrder = dfProductionOrder.groupby(['product_code', 'order_id' ,'production_order_id' , 'production_code',
                                    'quantity_production_order' ,
                                    'time_create' , 'completed_at' , 'time_stock_out'], as_index=False)['quantity_exported_production_order'].sum()

dfProductionOrder = dfProductionOrder.groupby(['product_code', 'order_id' ,'production_order_id' , 'production_code',
                                    'time_create' , 'completed_at' , 'time_stock_out' , 'quantity_exported_production_order'], as_index=False)['quantity_production_order'].sum()

dfProductionOrder


,product_code,order_id,production_order_id,production_code,time_create,completed_at,time_stock_out,quantity_exported_production_order,quantity_production_order
0,1-1035CW-ACG-00-UL,1398,1809,PYC-656-CAT,2022-10-17,2022-10-18,2022-10-18,329.95,329.95
1,1-1035CW-ACG-00-UL,1399,1809,PYC-656-CAT,2022-10-17,2022-10-18,2022-10-18,410.02,410.02
2,1-1035CW-ACG-L1-UL,1398,1809,PYC-656-CAT,2022-10-17,2022-10-18,2022-10-18,41.47,41.47
3,1-1035CW-ACG-L1-UL,1399,1809,PYC-656-CAT,2022-10-17,2022-10-18,2022-10-18,47.08,47.08
4,1-1035CW-SGY-00-UL,1400,1839,PYC-667-CAT,2022-10-18,2022-10-19,2022-10-18,110.48,110.48
...,...,...,...,...,...,...,...,...,...
36603,9-X44317-Y00-5L-BN,1911,1637,PYC-215-THUDUC,2022-10-03,2022-10-04,2022-10-03,1.00,1.00
36604,9-X44317-Y00-5L-BN,1912,1637,PYC-215-THUDUC,2022-10-03,2022-10-04,2022-10-03,1.00,1.00
36605,9-X44317-Y00-5L-BN,1913,1637,PYC-215-THUDUC,2022-10-03,2022-10-04,2022-10-03,1.00,1.00
36606,9-X44317-Y00-5L-BN,1963,1713,PYC-227-THUDUC,2022-10-07,2022-10-08,2022-10-08,0.50,0.50


In [11]:
dfOrder = dfOrder.groupby(['product_id', 'order_code' ,'order_id' ,
                                    'product_code'], as_index=False)['total_quota'].sum()

In [12]:
dfProductionOrder = dfProductionOrder.replace(np.NaN , 0)
dfProductionOrderMerge = pd.merge(dfOrder, dfProductionOrder , on = ['product_code' , 'order_id'], how = 'left')
dfProductionOrderMerge = dfProductionOrderMerge.replace(np.NaN , 0)

In [13]:
dfProductionOrderMerge[(dfProductionOrderMerge['order_code'] == 'AD-220623-YELIR 2')].head(2)

,product_id,order_code,order_id,product_code,total_quota,production_order_id,production_code,time_create,completed_at,time_stock_out,quantity_exported_production_order,quantity_production_order
336,82,AD-220623-YELIR 2,763,1-CVAC06-BLK-00-HV,96.0,499.0,PYC-194-CAT,2022-06-30,2022-07-01,2022-07-01,95.999999,96.0
337,82,AD-220623-YELIR 2,763,1-CVAC06-BLK-00-HV,96.0,499.0,PYC-194-CAT,2022-06-30,2022-07-01,2022-07-04,0.000001,96.0


In [14]:
# dfProductionOrderMerge[dfProductionOrderMerge['order_code'] == 'Charter Hat-Med-Cream-Sand']

In [ ]:
dfProductionOrderMerge = dfProductionOrderMerge.drop(columns= {'product_id' , 'order_id' ,'production_order_id'})
dfProductionOrderMerge.rename(columns = {'order_code':'Mã đơn hàng' , 'product_code': 'Mã NPL' , 'total_quota': 'Định mức đơn hàng' , 'production_code': 'Mã LSX' , 'quantity_production_order' : 'SL LSX' , 
                                         'quantity_exported_production_order' : 'Tổng đã xuất' , 'time_create': 'Ngày tạo LSX',
                                         'completed_at': 'Ngày hẹn LSX', 'time_stock_out': 'ngày xuất kho'}, inplace = True)
dfProductionOrderMerge

: 

In [ ]:
dfProductionOrderMerge[dfProductionOrderMerge['Mã NPL'] == '9-CCI166-000-00-TA']

: 

In [ ]:
dfProductionOrderMerge.to_excel('../data_complete/getProductionOrder.xlsx', index= False)

: 

#### Outsourcing Stock In


In [ ]:
dfProduct = pd.read_csv("../dataDA/Product.csv")

: 

In [ ]:
dfPrimary = dfProduct.loc[dfProduct['is_outsourcing'] == 0]
dfSecondary = dfProduct.loc[dfProduct['is_outsourcing'] == 1]

: 

In [ ]:
dfPrimary

: 

In [ ]:
dfProduct

: 

In [ ]:
dfOutsourcingStockIn = pd.read_csv('../dataDA/OutsourcingStockInBao.csv')
dfOutSourcingDetail = pd.read_csv('../dataDA/OutSourcingDetailBao.csv')

: 

In [ ]:
dfOutSourcingDetail[dfOutSourcingDetail['outsourcing_order_id'] == 138].shape

: 

In [ ]:
dfOutsourcingStockIn[dfOutsourcingStockIn['outsourcing_order_id'] == 138].shape

: 

In [ ]:
dfOutSourcingDetail.sort_values(by = ['outsourcing_order_id'])

: 

In [ ]:
dfOutsourcingStockIn

: 

In [ ]:
dfOutsourcingStockIn[dfOutsourcingStockIn['outsourcing_order_id'] == 295]

: 

In [ ]:
dfOutSourcingDetail

: 

In [ ]:
dfOutSourcingDetailMerge = dfOutSourcingDetail.merge(dfOutsourcingStockIn , on = ['outsourcing_order_id' , 
                                                            'product_id' , 'outsourcing_quantity'], how = 'right').replace(np.NaN , 0)
dfOutSourcingDetailMerge

: 

In [ ]:
dfProduct[dfProduct['product_id'] == 8617]

: 

In [ ]:
dfOutSourcingDetailMerge = dfOutSourcingDetailMerge.merge(dfProduct[['product_id' , 'product_code']] , on = 'product_id'  , how = 'left')
# dfOutSourcingDetailMerge

: 

In [ ]:
# dfOutSourcingDetailMerge[dfOutSourcingDetailMerge['outsourcing_order_id'] == 138]

: 

In [ ]:
dfOutSourcingDetailMerge = dfOutSourcingDetailMerge.drop(columns= {'product_id' , 'outsourcing_order_id'})
dfOutSourcingDetailMerge.rename(columns = {'code':'Mã đơn hàng' , 'name': 'Tên đơn hàng', 'product_code': 'Mã NPL' , 'is_complete_order': 'Hoàn tất đơn hàng' ,
                                         'is_complete_order_detail': 'Hoàn tất chi tiết đơn hàng' , 'outsourcing_quantity': 'Số lượng GCN' , 'quantity_stock_in' : 'Số lượng GCN nhập kho' , 
                                        'created_at_': 'Ngày tạo ĐH', 'delivery_at_': 'Ngày hẹn ĐH', 'delivery_real_': 'Ngày xuất kho GCN'}, inplace = True)
dfOutSourcingDetailMerge

: 

In [ ]:
dfOutSourcingDetailMerge.to_excel('../data_complete/getOutSourcingStockIn.xlsx', index= False)

: 

In [ ]:
dfOutSourcingDetailMerge[dfOutSourcingDetailMerge['Hoàn tất chi tiết đơn hàng'].isnull()]

: 

: 

: 

#### Outsourcing Stock Out

In [ ]:
dfOutSourcingMaterial = pd.read_csv('../dataDA/OutSourcingMaterialBao.csv')
dfOutSourcingMaterial = dfOutSourcingMaterial.replace(np.NaN , 0)
dfOutSourcingMaterial

: 

In [ ]:
dfProductNew  = dfProduct.rename(columns={"product_id": "outsourcing_product_id" , "product_code" : "outsourcing_product_code"})
dfOutSourcingMaterialMerge = dfOutSourcingMaterial.merge(dfProduct[['product_id' , 
                    'product_code']] , on = 'product_id' , how = 'left').merge(dfProductNew[['outsourcing_product_id' , 
                    'outsourcing_product_code']] , on = 'outsourcing_product_id' , how = 'left')
dfOutSourcingMaterialMerge

: 

In [ ]:
dfOutSourcingStockOut = pd.read_csv('../dataDA/OutSourcingStockOutBao.csv')
dfOutSourcingStockOut = dfOutSourcingStockOut.replace(np.NaN , 0)
dfOutSourcingStockOut

: 

In [ ]:
dfOutSourcingStockOut = dfOutSourcingStockOut.groupby(['product_id' , 'outsourcing_order_id' , 'is_exported',
                                                       'outsourcing_detail_id', 'date_of_inventory' ], as_index=False)['quantity_exported_outsourcing'].sum()
dfOutSourcingStockOut.sort_values(by = ['product_id'])
dfOutSourcingStockOut

: 

In [ ]:
dfOutSourcingMaterialMergeNew = dfOutSourcingMaterialMerge.merge(dfOutSourcingStockOut, on = ['product_id', 'outsourcing_detail_id' , 'outsourcing_order_id'
                                                        , 'is_exported'] , how = 'left')

dfOutSourcingMaterialMergeNew

: 

In [ ]:
dfOutSourcingMaterialMergeNew[dfOutSourcingMaterialMergeNew['outsourcing_order_id'] == 1375]

: 

In [ ]:
dfOutSourcingOrder = pd.read_csv('../dataDA/OutSourcingOrderBao.csv')
dfOutSourcingOrder

: 

In [ ]:
dfOutSourcingMaterialMerge = dfOutSourcingMaterialMergeNew.merge(dfOutSourcingOrder, on = ['outsourcing_order_id'] , how = 'left')
dfOutSourcingMaterialMerge.head(2)

: 

In [ ]:
dfRelationShip = pd.read_csv('../dataDA/Relationship.csv')
dfRelationShipOrder = dfRelationShip.rename(columns= {'product_code' : 'product_code_po'})
dfRelationShipOrder

: 

In [ ]:
dfOutSourcingMaterialRel = dfOutSourcingMaterialMerge.merge(dfRelationShipOrder , on = ['outsourcing_product_id' , 'product_id'] , how = 'left')
dfOutSourcingMaterialRel.head(2)

: 

In [ ]:
dfOutSourcingMaterialRel = dfOutSourcingMaterialRel.drop(columns= {'product_id' , 'outsourcing_order_id' , 'outsourcing_product_id' , 'outsourcing_order_id' , 'outsourcing_detail_id'})

new_col1 = ['code','name' , 'product_code' ,'outsourcing_product_code', 'total_quota', 
      'quantity_outsourcing_detail', 'is_complete_detail', 'is_exported',
      'is_complete_infos', 'quota_percent' , 'quota_outsourcing' , 'date_of_inventory',
      'quantity_exported_outsourcing' , 'product_code_po']
   
dfOutSourcingMaterialStockOut= dfOutSourcingMaterialRel.reindex(columns=new_col1)
   
dfOutSourcingMaterialStockOut.rename(columns = {'code':'Mã ĐH GCN' , 'name': 'Tên ĐH GCN', 'product_code': 'Mã WIP' , 'outsourcing_product_code': 'Mã Thành phẩm',
                                             'total_quota': 'SL WIP cần GCN', 'quantity_outsourcing_detail': 'SL thành phẩm cần', 'is_complete_detail': 'Nhập kho Thành phẩm',
                                        'is_exported': 'Đã Xuất WIP' , 'is_complete_infos': 'ĐH hoàn tất' , 'quota_percent': 'Định mức GCN' , 'date_of_inventory': 'Ngày xuất kho',
                                        'quantity_exported_outsourcing':'Số lượng WIP đi GCN thực tế' , 'quota_outsourcing' : 'Định mức theo thu mua' , 'product_code_po' : 'Mã NPL bán TP theo thu mua'}, inplace = True)
dfOutSourcingMaterialStockOut

: 

In [ ]:
dfOutSourcingMaterialStockOut.to_excel('../data_complete/getOutSourcingStockOut.xlsx', index= False)

: 

#### PYC nhap kho

: 

: 

: 